<a href="https://colab.research.google.com/github/HakureiPOI/cryptolabs_2024/blob/main/Lab4_EIGamal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HakureiPOI
***Lab4_EIGamal***

---
### EIGamal 类

In [7]:
import random
import hashlib

In [10]:
class ElGamal:
    def __init__(self):
        self.p = None
        self.g = None
        self.y = None
        self.__x = None
        self.public_key = None
        self.__private_key = None

    def __gcd(self, a, b):
        # 计算两个数的最大公约数（欧几里得算法）
        while b != 0:
            a, b = b, a % b
        return a

    def __extended_gcd(self, a, b):
        # 扩展欧几里得算法，用于求解模逆
        if a == 0:
            return b, 0, 1
        gcd, x1, y1 = self.__extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y

    def __mod_inverse(self, k, p):
        # 使用扩展欧几里得算法计算模逆
        gcd, x, _ = self.__extended_gcd(k, p)
        if gcd != 1:
            return None  # 如果最大公约数不是1，则模逆不存在
        return x % p

    def generate_keys(self, p, g):
        # 生成公钥和私钥
        self.p = p
        self.g = g
        self.__x = random.randint(1, p - 2)  # 私钥
        self.y = pow(g, self.__x, p)  # 公钥
        self.public_key = (self.p, self.g, self.y)
        self.__private_key = self.__x
        return self.public_key, self.__private_key

    def __hash_message(self, m):
        # 使用SHA-256对消息进行哈希
        hash_obj = hashlib.sha256()
        hash_obj.update(str(m).encode('utf-8'))
        return int(hash_obj.hexdigest(), 16)

    def sign_message(self, m):
        # 对消息进行签名
        while True:
            k = random.randint(1, self.p - 2)
            if self.__gcd(k, self.p - 1) == 1:
                break
        print(f"用于签名的随机数 k: {k}")

        r = pow(self.g, k, self.p)
        k_inv = self.__mod_inverse(k, self.p - 1)
        h_m = self.__hash_message(m)
        s = (k_inv * (h_m - self.__x * r)) % (self.p - 1)
        return r, s

    def verify_signature(self, m, r, s):
        # 验证签名的有效性
        if not (0 < r < self.p):
            return False

        h_m = self.__hash_message(m)
        v1 = (pow(self.y, r, self.p) * pow(r, s, self.p)) % self.p
        v2 = pow(self.g, h_m, self.p)
        return v1 == v2

---
### 测试

In [11]:
if __name__ == "__main__":
    # 素数 p 和生成元 g
    p = 467
    g = 2

    # 创建 ElGamal 实例
    elgamal = ElGamal()

    # 生成公钥和私钥
    public_key, private_key = elgamal.generate_keys(p, g)
    print("公钥:", public_key)
    print("私钥:", private_key)

    # 要签名的消息（使用学号）
    message = 220110720

    # 使用不同的随机数 k 进行两次签名
    print("\n第一次签名:")
    r1, s1 = elgamal.sign_message(message)
    print("签名: (r, s) =", (r1, s1))

    print("\n第二次签名:")
    r2, s2 = elgamal.sign_message(message)
    print("签名: (r, s) =", (r2, s2))

    # 验证签名
    print("\n验证第一次签名:")
    is_valid_1 = elgamal.verify_signature(message, r1, s1)
    print("第一次签名是否有效:", is_valid_1)

    print("\n验证第二次签名:")
    is_valid_2 = elgamal.verify_signature(message, r2, s2)
    print("第二次签名是否有效:", is_valid_2)

    # 验证消息被篡改的情况
    tampered_message = 220110721
    print("\n验证被篡改的消息:")
    is_valid_tampered = elgamal.verify_signature(tampered_message, r1, s1)
    print("篡改后的消息签名是否有效:", is_valid_tampered)

公钥: (467, 2, 118)
私钥: 39

第一次签名:
用于签名的随机数 k: 357
签名: (r, s) = (217, 216)

第二次签名:
用于签名的随机数 k: 275
签名: (r, s) = (457, 82)

验证第一次签名:
第一次签名是否有效: True

验证第二次签名:
第二次签名是否有效: True

验证被篡改的消息:
篡改后的消息签名是否有效: False
